<a href="https://colab.research.google.com/github/AndresPautrat/Concurrente/blob/main/go_Channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang

In [ ]:
%%writefile 1.go
package main
import(
    "fmt"
)

func main(){
    var ch chan string
    ch = make(chan string)

    go func(){
        ch<-"Hola,Mundo canalizado!"
    }()

    str:=<-ch
    fmt.Printf("Mensaje: %s\n",str)
}

Overwriting 1.go


In [ ]:
!go run 1.go

Mensaje: Hola,Mundo canalizado!


In [ ]:
%%writefile 2.go
package main
import(
    "fmt"
    "math/rand"
    "time"
)

func rng(id, n int, ch chan int){
    for i:=0; i <n;i++{
        ch<-id *100000+(i+1)*1000 +rand.Intn(100);
    }
}

func main(){
    rand.Seed(time.Now().UTC().UnixNano())
    var ch chan int
    ch = make(chan int)

    n1,n2 := 8,11
    go rng(1,n1,ch)
    go rng(2,n2,ch)

    for i:= 0; i<n1+n2; i++{
        fmt.Printf("Mensaje: %d\n",<-ch)
    }
}

Overwriting 2.go


In [ ]:
!go run 2.go

Mensaje: 201076
Mensaje: 202083
Mensaje: 101010
Mensaje: 102061
Mensaje: 103065
Mensaje: 104036
Mensaje: 105098
Mensaje: 203023
Mensaje: 106067
Mensaje: 204017
Mensaje: 205044
Mensaje: 206092
Mensaje: 207050
Mensaje: 208025
Mensaje: 209056
Mensaje: 210077
Mensaje: 211057
Mensaje: 107026
Mensaje: 108054


In [ ]:
%%writefile 3.go
package main
import(
    "fmt"
    "math/rand"
    "time"
)

func rng(id, n int, ch chan int){
    for i:=0; i <n;i++{
        ch<-fmt.Printf("Mensaje: %d \t i:%d \t Numero:%d", id, i, num)
    }
}

func main(){
    rand.Seed(time.Now().UTC().UnixNano())
    var ch chan string
    ch = make(chan string)

    n1,n2 := 8,11
    go rng(1,n1,ch)
    go rng(2,n2,ch)

    go func(){
        for msg := range ch1{
            fmt.Printf("Mensaje: %s\n",msg)
        }
    }()

    for msg:=range ch2{
        fmt.Printf("Mensaje: %d\n",<-ch)
    }
}

Writing 3.go


In [ ]:
!go run 3.go

# command-line-arguments
./3.go:10:23: multiple-value fmt.Printf() in single-value context
./3.go:10:67: undefined: num
./3.go:20:12: cannot use ch (type chan string) as type chan int in argument to rng
./3.go:21:12: cannot use ch (type chan string) as type chan int in argument to rng
./3.go:24:26: undefined: ch1
./3.go:29:20: undefined: ch2


In [ ]:
%%writefile 4.go
package main
import(
    "fmt"
)

const(
    MAX=9
    K=4
)

func compress(inC,pipe chan rune){
    n :=0
    previous:=inC
    for c:=range inC{
        if c == previous && n<MAX-1{
            n++
        } else{
            if n >0{
                pipe<-rune(n+1+48)
                n=0
            }
            pipe<-previous
            previous=c
        }   
    }
    close(pipe)
}

func output(pipe,outC chan rune){
    m:=0
    for c:= range pipe{
        out<-c
        m++
        if m==K{
            outC<-"\n"
            m=0
        }
    }
    close(outC)
}

func main(){
    inC:= make(chan rune)
    pipe:=make (chan rune)
    outC:=make(chan rune)

    go compress(inC,pipe)
    go output(pipe, outC)

    go func(){
        str:="asdasdksadfjansdkjasfdasfasddf."
        for _, chr :=range str{
            inC<-chr
        }
        close(inC)
    }()

    for c:=range outC{
        fmt.Printf("%c",c)
    }
    max :=9
    k:=4
    var inC chan int
    var pipe chan int
    var outC chan int
}

Writing 4.go


In [ ]:
!go run 4.go

# command-line-arguments
./4.go:15:14: invalid operation: c == previous (mismatched types rune and chan rune)
./4.go:22:17: cannot use previous (type chan rune) as type rune in send
./4.go:23:21: cannot use c (type rune) as type chan rune in assignment
./4.go:32:9: undefined: out
./4.go:35:19: cannot convert "\n" (type untyped string) to type rune
./4.go:63:9: inC redeclared in this block
	previous declaration at ./4.go:43:5
./4.go:64:9: pipe redeclared in this block
	previous declaration at ./4.go:44:5
./4.go:65:9: outC redeclared in this block
	previous declaration at ./4.go:45:5


In [ ]:
%%writefile multiplier.go
package main

import (
	"fmt"
)

var end chan bool

func zero(n int, west chan float64) {
	for i := 0; i < n; i++ {
		west <- 0.0
	}
	close(west)
}

func source(row []float64, south chan float64) {
	for _, element := range row {
		south <- element
	}
	close(south)
}

func sink(north chan float64) {
	for range north {
	}
}

func result(c [][]float64, i int, east chan float64) {
	j := 0
	for element := range east {
		c[i][j] = element
		j++
	}
	end <- true
}

func multiplier(firstElement float64, north, east, south, west chan float64) {
	for secondElement := range north {
		sum := <-east
		sum = sum + firstElement*secondElement
		south <- secondElement
		west <- sum
	}
	close(south)
	close(west)
}

func main() {
	a := [][]float64{{1, 2, 3,2}, {4, 5, 6,3}, {7, 8, 9,1}, {7, 8, 9,1}}
	b := [][]float64{{1, 0, 2,1}, {0, 1, 2,0}, {1, 0, 0,2}, {7, 8, 9,1}}
	c := [][]float64{{0, 0, 0,0}, {0, 0, 0,0}, {0, 0, 0,0},{0, 0, 0,0}}
	end = make(chan bool)
	nra := len(a)
	nca := len(a[0])
	ns := make([][]chan float64, nra+1) // canales norte sur, matrix de 4x3
	for i := range ns {
		ns[i] = make([]chan float64, nca)
		for j := range ns[i] {
			ns[i][j] = make(chan float64)
		}
	}
	ew := make([][]chan float64, nra) // canales easte oeste, matrix de 3x4
	for i := range ew {
		ew[i] = make([]chan float64, nca+1)
		for j := range ew[i] {
			ew[i][j] = make(chan float64)
		}
	}
	for i := 0; i < nra; i++ {
		go zero(nra, ew[i][nca])
		go result(c, i, ew[i][0])
	}
	for i := 0; i < nca; i++ {
		go source(b[i], ns[0][i])
		go sink(ns[nra][i])
	}
	for i := 0; i < nra; i++ {
		for j := 0; j < nca; j++ {
			go multiplier(a[i][j],
				ns[i][j], ew[i][j+1],
				ns[i+1][j], ew[i][j])
		}
	}
	for i := 0; i < nra; i++ {
		<-end
	}
	fmt.Println(c)
}

Overwriting multiplier.go


In [ ]:
!go run multiplier.go

[[18 18 24 9] [31 29 45 19] [23 16 39 26] [23 16 39 26]]
